# Training

In [ ]:
import os
from pathlib import Path

def create_training_files(run_number, astropy_table, crop_to=None, output_dir=None):
      """
      Create training configuration files for a Hyrax run with 8 variations.
      
      Parameters:
      -----------
      run_number : int
          The run number (e.g., 4 for run4)
      astropy_table : str
          Path to the astropy table FITS file
      crop_to : list or tuple, optional
          Image crop dimensions [width, height]. Defaults to [150, 150]
      output_dir : str, optional
          Output directory. If None, uses the standard pattern.
      """
      if output_dir is None:
          output_dir = f"/work/hdd/bemi/dmiura/800batch/hyrax_runs/run{run_number}"

      if crop_to is None:
          crop_to = [50, 50] # TODO: Check if this is a good value

      # Create output directory if it doesn't exist
      Path(output_dir).mkdir(parents=True, exist_ok=True)

      # Configuration patterns for files 1-8
      configs = [
          # Files 1-4: SGD optimizer with different learning rates
          {"optimizer": "torch.optim.SGD", "lr": 0.01, "momentum": True},    # _1
          {"optimizer": "torch.optim.SGD", "lr": 0.1, "momentum": True},     # _2
          {"optimizer": "torch.optim.SGD", "lr": 0.001, "momentum": True},   # _3
          {"optimizer": "torch.optim.SGD", "lr": 0.0001, "momentum": True},  # _4
          # Files 5-8: Adam optimizer with different learning rates
          {"optimizer": "torch.optim.Adam", "lr": 0.01, "momentum": False},   # _5
          {"optimizer": "torch.optim.Adam", "lr": 0.1, "momentum": False},    # _6
          {"optimizer": "torch.optim.Adam", "lr": 0.001, "momentum": False},  # _7
          {"optimizer": "torch.optim.Adam", "lr": 0.0001, "momentum": False}, # _8
      ]

      for i, config in enumerate(configs, 1):
          create_toml_file(run_number, i, config, astropy_table, crop_to, output_dir)
          create_job_file(run_number, i, output_dir)

def create_toml_file(run_number, file_number, config, astropy_table, crop_to, output_dir):
      """Create a TOML configuration file."""
      optimizer_name = config["optimizer"]
      lr = config["lr"]
      has_momentum = config["momentum"]

      # Build optimizer section
      optimizer_section = f"""[optimizer]
name = "{optimizer_name}"
["{optimizer_name}"]
lr = {lr}"""

      if has_momentum:
          optimizer_section += "\nmomentum = 0.9"

      toml_content = f"""[general]
dev_mode = false
log_destination = "stderr"
log_level = "info" 
data_dir = "/work/hdd/bemi/dmiura/800batch/data_download/split_images"
results_dir = "/work/hdd/bemi/dmiura/800batch/hyrax_runs/results/"

##########################################

[model]
name = "HyraxAutoencoder"
base_channel_size = 32
latent_dim = 64
final_layer = "tanh"

[criterion]
name = "torch.nn.MSELoss"
band_loss_reduction = "mean"

{optimizer_section}
      
[train]
weights_filepath = "example_model.pth"
epochs = 20
resume = false
split = "train"
experiment_name = "run{run_number}"
run_name = "run{run_number}_{file_number}"


##########################################

[data_set]
# Name of the built-in data loader to use or the import path to an external data
# loader. e.g. "HSCDataSet", "user_pkg.data_set.ExternalDataSet"
name = "FitsImageDataSet"

# Crop pixel dimensions for images, e.g., [100, 100]. If false, scans for the
# smallest image size in [general].data_dir and uses it.
crop_to = {crop_to}

# Used by HSCDataSet, LSSTDataset, and DownloadedLSSTDataset. 
# Limit to only particular filters. When `false`, use all filters. 
# Options: ["HSC-G", "HSC-R", "HSC-I", "HSC-Z", "HSC-Y"] for HSC
# Options: ["u", "g", "r", "i", "z" , "y"] for LSST
filters = false

# Path to a fits file that specifies object IDs to use from the data stored in
# [general].data_dir. Implementation is data_set class dependent. Use `false` for no filtering.
filter_catalog = "/work/hdd/bemi/dmiura/800batch/data_download/800x5_catalog.fits"

# The transformation to be applied to images before being passed on to the model
# This must be a valid Numpy function. Passing false will result in no transformations
# (other than cropping) be applied to the images.  
transform = "tanh"

# train_size, validation_size, and test_size use these conventions:
# * A `float` between `0.0` and `1.0` is the proportion of the dataset to include in the split.
# * An `int`, represents the absolute number of samples in the particular split.
# * It is an error for these values to add to more than 1.0 as ratios or the size
#   of the dataset if expressed as integers.

# Size of the train split. If `false`, the value is automatically set to the
# complement of test_size plus validate_size (if any).
train_size = 0.8

# Size of the validation split. If `false`, and both train_size and test_size
# are defined, the value is set to the complement of the other two sizes summed.
# If `false`, and only one of the other sizes is defined, no validate split is created.
validate_size = 0.1

# Size of the test split. If `false`, the value is set to the complement of train_size plus
# the validate_size (if any). If `false` and `train_size = false`, test_size is set to `0.25`.
test_size = 0.1

# Number to seed with for generating a random split. Use `false` to seed from a
# system source at runtime.
seed = 1

# If `true`, cache samples in memory during training to reduce runtime after the
# first epoch. Set to `false` when running inference or on memory-constrained systems.
use_cache = true

# If `true`, preload the in memory cache using many worker threads when the dataset is constructed 
# to reduce the effect of filesystem latency on first epoch runtime. 
# Warning: Only suitable for situations where the entire dataset fits in system memory
preload_cache = true

# Override the name of the object_id column for FitsImageDataset, HSCDataset and DownloadedLSSTDataset
object_id_column_name = "object_id"

# Override the name of the filter column for FitsImageDataset and HSCDataset
filter_column_name = false

# Override the name of the filename column for FitsImageDataset and HSCDataset
filename_column_name = false

# Replace NaN in input data with a value, modes are false for no replacement or "quantile" to replace with a 
# defined quantile of the non-NaN data, see nan_quantile.
nan_mode = false

# When replacing NaN values with a quantile, which quantile in the non-nan tensor should be used.
nan_quantile = 0.05

# The astropy table to use as a catalog in LSSTDataSet and friends
astropy_table = "{astropy_table}"
***

[data_loader]
batch_size = 256
########################################"""

      filename = f"{output_dir}/train{run_number}_{file_number}.toml"
      with open(filename, 'w') as f:
          f.write(toml_content)
      print(f"Created {filename}")

def create_job_file(run_number, file_number, output_dir):
      """Create a SLURM job file."""
      job_content = f"""#!/bin/bash
#
#SBATCH --job-name=t{run_number}_{file_number}
#SBATCH --output=train{run_number}_{file_number}.txt
#SBATCH --partition=gpuA40x4
#SBATCH --mem=50G
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1  # could be 1 for py-torch
#SBATCH --cpus-per-task=16   # spread out to use 1 core per numa, set to 64 if tasks is 1
#SBATCH --gpus-per-node=1
#SBATCH --account=bemi-delta-gpu    # <- match to a "Project" returned by the "accounts" command
#SBATCH -t 01:00:00

hyrax train --runtime-config=/work/hdd/bemi/dmiura/800batch/hyrax_runs/run{run_number}/train{run_number}_{file_number}.toml"""

      filename = f"{output_dir}/train{run_number}_{file_number}.sh"
      with open(filename, 'w') as f:
          f.write(job_content)
      print(f"Created {filename}")

In [ ]:
create_training_files(
          run_number=7,
          astropy_table="/mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_20.fits",
          crop_to=[150, 150]
      )

Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run7/train7_1.toml
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run7/train7_1.job
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run7/train7_2.toml
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run7/train7_2.job
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run7/train7_3.toml
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run7/train7_3.job
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run7/train7_4.toml
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run7/train7_4.job
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run7/train7_5.toml
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run7/train7_5.job
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run7/train7_6.toml
Crea

In [4]:
create_training_files(
          run_number=8,
          astropy_table="/mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_20_22.fits",
          crop_to=[70, 70]
      )

Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run8/train8_1.toml
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run8/train8_1.job
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run8/train8_2.toml
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run8/train8_2.job
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run8/train8_3.toml
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run8/train8_3.job
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run8/train8_4.toml
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run8/train8_4.job
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run8/train8_5.toml
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run8/train8_5.job
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run8/train8_6.toml
Crea

In [5]:
create_training_files(
          run_number=9,
          astropy_table="/mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_22_24.fits",
          crop_to=[50, 50]
      )

Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run9/train9_1.toml
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run9/train9_1.job
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run9/train9_2.toml
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run9/train9_2.job
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run9/train9_3.toml
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run9/train9_3.job
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run9/train9_4.toml
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run9/train9_4.job
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run9/train9_5.toml
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run9/train9_5.job
Created /mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run9/train9_6.toml
Crea

In [ ]:
import os
import subprocess
import time

def submit_jobs(run_number, start_num, end_num):
      """
      Submit job scripts for a range of numbers.
      
      Parameters:
      -----------
      run_number : int
          The run number (e.g., 4 for run4)
      start_num : int
          Starting job number
      end_num : int
          Ending job number
      """
      # Go to the run directory
      run_dir = f"/work/hdd/bemi/dmiura/800batch/hyrax_runs/run{run_number}"
      os.chdir(run_dir)

      # Submit each job in the range
      for i in range(start_num, end_num + 1):
          job_file = f"train{run_number}_{i}.sh"
          cmd = f"sbatch {job_file}"
          print(f"Submitting: {cmd}")
          os.system(cmd)
          if i == 1: #Trying to prevent MLFLow duplicate naming bug
              time.sleep(10)

In [17]:
run = True
if run is True:
    submit_jobs(9,9,17)

Submitting: sbatch train9_9.job
Submitted batch job 27612839
Submitting: sbatch train9_10.job
Submitted batch job 27612840
Submitting: sbatch train9_11.job
Submitted batch job 27612841
Submitting: sbatch train9_12.job
Submitted batch job 27612842
Submitting: sbatch train9_13.job
Submitted batch job 27612843
Submitting: sbatch train9_14.job
Submitted batch job 27612844
Submitting: sbatch train9_15.job
Submitted batch job 27612845
Submitting: sbatch train9_16.job
Submitted batch job 27612846
Submitting: sbatch train9_17.job
Submitted batch job 27612847


# Inference

In [9]:
import os
import re
from pathlib import Path

def extract_model_directory(train_output_file):
  """Extract model directory from train output file."""
  try:
      with open(train_output_file, 'r') as f:
          content = f.read()

      # Look for the checkpoint save pattern
      pattern = r'Latest checkpoint saved as: (.+)/checkpoint_epoch_\d+\.pt'
      match = re.search(pattern, content)

      if match:
          model_dir = match.group(1)
          return f"{model_dir}/example_model.pth"
      else:
          raise ValueError(f"Could not find checkpoint path in {train_output_file}")

  except FileNotFoundError:
      raise FileNotFoundError(f"Training output file not found: {train_output_file}")

def create_infer_scripts_batch(x, y_values, base_directory="."):
  """Generate infer scripts for multiple trainx_y configurations."""

  # Append runx to the base directory path
  run_dir = Path(base_directory) / f"run{x}"
  results = []

  for y in y_values:
      train_name = f"train{x}_{y}"
      infer_name = f"infer{x}_{y}"

      # File paths
      train_toml = run_dir / f"{train_name}.toml"
      train_job = run_dir / f"{train_name}.job"
      train_output = run_dir / f"{train_name}.txt"

      # Check if training files exist
      if not train_toml.exists():
          print(f"Warning: {train_toml} not found, skipping...")
          continue

      # Extract model weights path from training output
      try:
          model_weights_path = extract_model_directory(train_output)
      except (FileNotFoundError, ValueError) as e:
          print(f"Error processing {train_name}: {e}")
          continue

      # Create infer config
      infer_toml = run_dir / f"{infer_name}.toml"
      with open(train_toml, 'r') as f:
          config_content = f.read()

      # Add infer section
      config_content += f"""

[infer]
model_weights_file = "{model_weights_path}"
split = false"""

      with open(infer_toml, 'w') as f:
          f.write(config_content)

      # Create infer job script
      if train_job.exists():
          infer_job = run_dir / f"{infer_name}.job"
          with open(train_job, 'r') as f:
              job_content = f.read()

          # Update job script parameters
          job_content = re.sub(rf'--job-name=t{x}_{y}', f'--job-name=i{x}_{y}', job_content)
          job_content = re.sub(rf'--output=train{x}_{y}\.txt', f'--output=infer{x}_{y}.txt', job_content)
          job_content = re.sub(rf'train{x}_{y}\.toml', f'infer{x}_{y}.toml', job_content)
          job_content = re.sub(r'hyrax train', 'hyrax infer', job_content)

          with open(infer_job, 'w') as f:
              f.write(job_content)

          results.append((infer_toml, infer_job))
          print(f"Created infer scripts for {train_name}")
      else:
          print(f"Warning: {train_job} not found for {train_name}")

  return results

In [3]:
base_directory = "/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs"
create_infer_scripts_batch(4, [8], base_directory=base_directory)

Created infer scripts for train4_8


[(PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/infer4_8.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/infer4_8.job'))]

In [10]:
base_directory = "/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs"
create_infer_scripts_batch(4, [10,12,13,14,15,18], base_directory=base_directory)

Created infer scripts for train4_10
Created infer scripts for train4_12
Created infer scripts for train4_13
Created infer scripts for train4_14
Created infer scripts for train4_15
Created infer scripts for train4_18


[(PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/infer4_10.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/infer4_10.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/infer4_12.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/infer4_12.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/infer4_13.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/infer4_13.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/infer4_14.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/infer4_14.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/infer4_15.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/infer4_15.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/infer4_18.toml'),
  PosixPath('/mmfs1/gscratch/dir

In [11]:
base_directory = "/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs"
create_infer_scripts_batch(5, [7,10,12,13,14,15], base_directory=base_directory)

Created infer scripts for train5_7
Created infer scripts for train5_10
Created infer scripts for train5_12
Created infer scripts for train5_13
Created infer scripts for train5_14
Created infer scripts for train5_15


[(PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/infer5_7.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/infer5_7.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/infer5_10.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/infer5_10.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/infer5_12.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/infer5_12.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/infer5_13.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/infer5_13.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/infer5_14.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/infer5_14.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/infer5_15.toml'),
  PosixPath('/mmfs1/gscratch/dirac

In [12]:
base_directory = "/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs"
create_infer_scripts_batch(6, [7,10,12,13,14,15], base_directory=base_directory)

Created infer scripts for train6_7
Created infer scripts for train6_10
Created infer scripts for train6_12
Created infer scripts for train6_13
Created infer scripts for train6_14
Created infer scripts for train6_15


[(PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/infer6_7.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/infer6_7.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/infer6_10.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/infer6_10.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/infer6_12.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/infer6_12.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/infer6_13.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/infer6_13.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/infer6_14.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/infer6_14.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/infer6_15.toml'),
  PosixPath('/mmfs1/gscratch/dirac

In [20]:
base_directory = "/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs"
create_infer_scripts_batch(7, [7,10,12,13,14,15,17,18], base_directory=base_directory)

Created infer scripts for train7_7
Created infer scripts for train7_10
Created infer scripts for train7_12
Created infer scripts for train7_13
Created infer scripts for train7_14
Created infer scripts for train7_15
Created infer scripts for train7_17
Created infer scripts for train7_18


[(PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/infer7_7.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/infer7_7.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/infer7_10.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/infer7_10.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/infer7_12.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/infer7_12.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/infer7_13.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/infer7_13.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/infer7_14.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/infer7_14.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/infer7_15.toml'),
  PosixPath('/mmfs1/gscratch/dirac

In [21]:
base_directory = "/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs"
create_infer_scripts_batch(8, [7,10,12,13,14,15,17], base_directory=base_directory)

Created infer scripts for train8_7
Created infer scripts for train8_10
Created infer scripts for train8_12
Created infer scripts for train8_13
Created infer scripts for train8_14
Created infer scripts for train8_15
Created infer scripts for train8_17


[(PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/infer8_7.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/infer8_7.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/infer8_10.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/infer8_10.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/infer8_12.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/infer8_12.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/infer8_13.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/infer8_13.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/infer8_14.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/infer8_14.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/infer8_15.toml'),
  PosixPath('/mmfs1/gscratch/dirac

In [22]:
base_directory = "/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs"
create_infer_scripts_batch(9, [7,10,12,13,14,15,17], base_directory=base_directory)

Created infer scripts for train9_7
Created infer scripts for train9_10
Created infer scripts for train9_12
Created infer scripts for train9_13
Created infer scripts for train9_14
Created infer scripts for train9_15
Created infer scripts for train9_17


[(PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/infer9_7.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/infer9_7.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/infer9_10.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/infer9_10.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/infer9_12.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/infer9_12.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/infer9_13.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/infer9_13.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/infer9_14.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/infer9_14.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/infer9_15.toml'),
  PosixPath('/mmfs1/gscratch/dirac

In [16]:
def submit_jobs(run_number, job_numbers):
      """
      Submit job scripts for specified job numbers.
      
      Parameters:
      -----------
      run_number : int
          The run number (e.g., 4 for run4)
      job_numbers : list
          List of job numbers to submit (e.g., [1, 3, 5, 7])
      """
      # Go to the run directory
      run_dir = f"/mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run{run_number}"
      os.chdir(run_dir)

      # Submit each job in the array
      for i in job_numbers:
          job_file = f"infer{run_number}_{i}.job"
          cmd = f"sbatch {job_file}"
          print(f"Submitting: {cmd}")
          os.system(cmd)

In [17]:
submit_jobs(4, [7,10,12,13,14,15,18])

Submitting: sbatch infer4_7.job
Submitted batch job 27646797
Submitting: sbatch infer4_10.job
Submitted batch job 27646799
Submitting: sbatch infer4_12.job
Submitted batch job 27646800
Submitting: sbatch infer4_13.job
Submitted batch job 27646801
Submitting: sbatch infer4_14.job
Submitted batch job 27646802
Submitting: sbatch infer4_15.job
Submitted batch job 27646803
Submitting: sbatch infer4_18.job
Submitted batch job 27646804


In [18]:
submit_jobs(5, [7,10,12,13,14,15])

Submitting: sbatch infer5_7.job
Submitted batch job 27646813
Submitting: sbatch infer5_10.job
Submitted batch job 27646814
Submitting: sbatch infer5_12.job
Submitted batch job 27646815
Submitting: sbatch infer5_13.job
Submitted batch job 27646816
Submitting: sbatch infer5_14.job
Submitted batch job 27646817
Submitting: sbatch infer5_15.job
Submitted batch job 27646818


In [19]:
submit_jobs(6, [7,10,12,13,14,15])

Submitting: sbatch infer6_7.job
Submitted batch job 27646819
Submitting: sbatch infer6_10.job
Submitted batch job 27646820
Submitting: sbatch infer6_12.job
Submitted batch job 27646821
Submitting: sbatch infer6_13.job
Submitted batch job 27646822
Submitting: sbatch infer6_14.job
Submitted batch job 27646823
Submitting: sbatch infer6_15.job
Submitted batch job 27646824


In [23]:
submit_jobs(7, [7,10,12,13,14,15,17,18])

Submitting: sbatch infer7_7.job
Submitted batch job 27646853
Submitting: sbatch infer7_10.job
Submitted batch job 27646854
Submitting: sbatch infer7_12.job
Submitted batch job 27646855
Submitting: sbatch infer7_13.job
Submitted batch job 27646856
Submitting: sbatch infer7_14.job
Submitted batch job 27646857
Submitting: sbatch infer7_15.job
Submitted batch job 27646858
Submitting: sbatch infer7_17.job
Submitted batch job 27646859
Submitting: sbatch infer7_18.job
Submitted batch job 27646861


In [25]:
submit_jobs(8, [7,10,12,13,14,15,17])

Submitting: sbatch infer8_7.job
Submitted batch job 27646885
Submitting: sbatch infer8_10.job
Submitted batch job 27646886
Submitting: sbatch infer8_12.job
Submitted batch job 27646887
Submitting: sbatch infer8_13.job
Submitted batch job 27646888
Submitting: sbatch infer8_14.job
Submitted batch job 27646889
Submitting: sbatch infer8_15.job
Submitted batch job 27646890
Submitting: sbatch infer8_17.job
Submitted batch job 27646891


In [26]:
submit_jobs(9, [7,10,12,13,14,15,17])

Submitting: sbatch infer9_7.job
Submitted batch job 27646894
Submitting: sbatch infer9_10.job
Submitted batch job 27646895
Submitting: sbatch infer9_12.job
Submitted batch job 27646896
Submitting: sbatch infer9_13.job
Submitted batch job 27646897
Submitting: sbatch infer9_14.job
Submitted batch job 27646898
Submitting: sbatch infer9_15.job
Submitted batch job 27646899
Submitting: sbatch infer9_17.job
Submitted batch job 27646900


# UMAP and VectorDB

In [2]:
import os
import re
from pathlib import Path

def extract_inference_directory(infer_output_file):
  """Extract inference directory from infer output file."""
  try:
      with open(infer_output_file, 'r') as f:
          content = f.read()

      # Look for the inference results save pattern
      pattern = r'Saving inference results at: (.+)'
      match = re.search(pattern, content)

      if match:
          inference_dir = match.group(1).strip()
          # Ensure it ends with / for consistency
          if not inference_dir.endswith('/'):
              inference_dir += '/'
          return inference_dir
      else:
          raise ValueError(f"Could not find inference directory in {infer_output_file}")

  except FileNotFoundError:
      raise FileNotFoundError(f"Inference output file not found: {infer_output_file}")

def create_udb_scripts_batch(x, y_values, base_directory="."):
  """Generate UDB scripts from infer configurations."""

  # Append runx to the base directory path
  run_dir = Path(base_directory) / f"run{x}"
  results = []

  for y in y_values:
      infer_name = f"infer{x}_{y}"
      udb_name = f"udb{x}_{y}"

      # File paths
      infer_toml = run_dir / f"{infer_name}.toml"
      infer_job = run_dir / f"{infer_name}.job"
      infer_output = run_dir / f"{infer_name}.txt"

      # Check if inference files exist
      if not infer_toml.exists():
          print(f"Warning: {infer_toml} not found, skipping...")
          continue

      # Extract inference directory from inference output
      try:
          inference_dir = extract_inference_directory(infer_output)
      except (FileNotFoundError, ValueError) as e:
          print(f"Error processing {infer_name}: {e}")
          continue

      # Create UDB config
      udb_toml = run_dir / f"{udb_name}.toml"
      with open(infer_toml, 'r') as f:
          config_content = f.read()

      # Add UDB-specific sections
      udb_sections = f"""
[results]
# Path to inference results to use for visualization and lookups. Uses latest inference run if none provided.
inference_dir = "{inference_dir}"

[vector_db]
name = "chromadb"
infer_results_dir = "{inference_dir}"


[umap]
# Number of data points used to fit the umap transform.
fit_sample_size = 5000

# Save the fitted umap as a pickle file 
save_fit_umap = false

# Use multiprocessing during transforming to umap space (More memory intensive)
parallel = true

# Name of the umap implementation to use
name = "umap.UMAP"


["umap.UMAP"]
# Specify any parameter accepted by https://umap-learn.readthedocs.io/en/latest/api.html#umap
# Dimension of the embedded space
n_components = 2

# Controls how UMAP balances local versus global structure in the data.
# See official documentation for details.
n_neighbors = 15
"""

      config_content += udb_sections

      with open(udb_toml, 'w') as f:
          f.write(config_content)

      # Create UDB job script
      if infer_job.exists():
          udb_job = run_dir / f"{udb_name}.job"
          with open(infer_job, 'r') as f:
              job_content = f.read()

          # Update job script parameters
          job_content = re.sub(rf'--job-name=i{x}_{y}', f'--job-name=u{x}_{y}', job_content)
          job_content = re.sub(rf'--output=infer{x}_{y}\.txt', f'--output=udb{x}_{y}.txt', job_content)
          job_content = re.sub(rf'infer{x}_{y}\.toml', f'udb{x}_{y}.toml', job_content)

          # Replace the hyrax command with umap and save_to_database commands
          # Remove the trailing directory from inference_dir for --input-dir parameter
          input_dir = inference_dir.rstrip('/')

          job_content = re.sub(
              r'hyrax infer --runtime-config=([^\s]+)',
              rf'hyrax umap --runtime-config=\1 --input-dir={input_dir}/\nhyrax save_to_database --runtime-config=\1 --input-dir={input_dir}/',
              job_content
          )

          with open(udb_job, 'w') as f:
              f.write(job_content)

          results.append((udb_toml, udb_job))
          print(f"Created UDB scripts for {infer_name}")
      else:
          print(f"Warning: {infer_job} not found for {infer_name}")

  return results


In [3]:
base_directory = "/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs"
create_udb_scripts_batch(4, [7,10,12,13,14,15,18], base_directory=base_directory)

Created UDB scripts for infer4_7
Created UDB scripts for infer4_10
Created UDB scripts for infer4_12
Created UDB scripts for infer4_13
Created UDB scripts for infer4_14
Created UDB scripts for infer4_15
Created UDB scripts for infer4_18


[(PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/udb4_7.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/udb4_7.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/udb4_10.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/udb4_10.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/udb4_12.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/udb4_12.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/udb4_13.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/udb4_13.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/udb4_14.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/udb4_14.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/udb4_15.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_

In [4]:
base_directory = "/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs"
create_udb_scripts_batch(5, [7,10,12,13,14,15], base_directory=base_directory)

Created UDB scripts for infer5_7
Created UDB scripts for infer5_10
Created UDB scripts for infer5_12
Created UDB scripts for infer5_13
Created UDB scripts for infer5_14
Created UDB scripts for infer5_15


[(PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/udb5_7.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/udb5_7.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/udb5_10.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/udb5_10.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/udb5_12.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/udb5_12.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/udb5_13.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/udb5_13.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/udb5_14.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/udb5_14.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/udb5_15.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_

In [5]:
base_directory = "/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs"
create_udb_scripts_batch(6, [7,10,12,13,14,15], base_directory=base_directory)

Created UDB scripts for infer6_7
Created UDB scripts for infer6_10
Created UDB scripts for infer6_12
Created UDB scripts for infer6_13
Created UDB scripts for infer6_14
Created UDB scripts for infer6_15


[(PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/udb6_7.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/udb6_7.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/udb6_10.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/udb6_10.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/udb6_12.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/udb6_12.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/udb6_13.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/udb6_13.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/udb6_14.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/udb6_14.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/udb6_15.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_

In [6]:
base_directory = "/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs"
create_udb_scripts_batch(7, [7,10,12,13,14,15,17,18], base_directory=base_directory)

Created UDB scripts for infer7_7
Created UDB scripts for infer7_10
Created UDB scripts for infer7_12
Created UDB scripts for infer7_13
Created UDB scripts for infer7_14
Created UDB scripts for infer7_15
Created UDB scripts for infer7_17
Created UDB scripts for infer7_18


[(PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/udb7_7.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/udb7_7.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/udb7_10.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/udb7_10.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/udb7_12.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/udb7_12.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/udb7_13.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/udb7_13.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/udb7_14.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/udb7_14.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/udb7_15.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_

In [7]:
base_directory = "/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs"
create_udb_scripts_batch(8, [7,10,12,13,14,15,17], base_directory=base_directory)

Created UDB scripts for infer8_7
Created UDB scripts for infer8_10
Created UDB scripts for infer8_12
Created UDB scripts for infer8_13
Created UDB scripts for infer8_14
Created UDB scripts for infer8_15
Created UDB scripts for infer8_17


[(PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/udb8_7.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/udb8_7.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/udb8_10.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/udb8_10.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/udb8_12.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/udb8_12.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/udb8_13.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/udb8_13.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/udb8_14.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/udb8_14.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/udb8_15.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_

In [8]:
base_directory = "/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs"
create_udb_scripts_batch(9, [7,10,12,13,14,15,17], base_directory=base_directory)

Created UDB scripts for infer9_7
Created UDB scripts for infer9_10
Created UDB scripts for infer9_12
Created UDB scripts for infer9_13
Created UDB scripts for infer9_14
Created UDB scripts for infer9_15
Created UDB scripts for infer9_17


[(PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/udb9_7.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/udb9_7.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/udb9_10.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/udb9_10.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/udb9_12.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/udb9_12.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/udb9_13.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/udb9_13.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/udb9_14.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/udb9_14.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/udb9_15.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_

In [3]:
import os

def submit_jobs(run_number, job_numbers):
      """
      Submit job scripts for specified job numbers.
      
      Parameters:
      -----------
      run_number : int
          The run number (e.g., 4 for run4)
      job_numbers : list
          List of job numbers to submit (e.g., [1, 3, 5, 7])
      """
      # Go to the run directory
      run_dir = f"/mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run{run_number}"
      os.chdir(run_dir)

      # Submit each job in the array
      for i in job_numbers:
          job_file = f"udb{run_number}_{i}.job"
          cmd = f"sbatch {job_file}"
          print(f"Submitting: {cmd}")
          os.system(cmd)


In [4]:
submit_jobs(4,[7,10,12,13,14,15,18])

Submitting: sbatch udb4_7.job
Submitted batch job 27664389
Submitting: sbatch udb4_10.job
Submitted batch job 27664390
Submitting: sbatch udb4_12.job
Submitted batch job 27664391
Submitting: sbatch udb4_13.job
Submitted batch job 27664392
Submitting: sbatch udb4_14.job
Submitted batch job 27664393
Submitting: sbatch udb4_15.job
Submitted batch job 27664394
Submitting: sbatch udb4_18.job
Submitted batch job 27664395


In [5]:
submit_jobs(5,[7,10,12,13,14,15])

Submitting: sbatch udb5_7.job
Submitted batch job 27664397
Submitting: sbatch udb5_10.job
Submitted batch job 27664398
Submitting: sbatch udb5_12.job
Submitted batch job 27664399
Submitting: sbatch udb5_13.job
Submitted batch job 27664400
Submitting: sbatch udb5_14.job
Submitted batch job 27664401
Submitting: sbatch udb5_15.job
Submitted batch job 27664402


In [6]:
submit_jobs(6,[7,10,12,13,14,15])

Submitting: sbatch udb6_7.job
Submitted batch job 27664403
Submitting: sbatch udb6_10.job
Submitted batch job 27664404
Submitting: sbatch udb6_12.job
Submitted batch job 27664405
Submitting: sbatch udb6_13.job
Submitted batch job 27664406
Submitting: sbatch udb6_14.job
Submitted batch job 27664407
Submitting: sbatch udb6_15.job
Submitted batch job 27664408


In [7]:
submit_jobs(7, [7,10,12,13,14,15,17,18])

Submitting: sbatch udb7_7.job
Submitted batch job 27664409
Submitting: sbatch udb7_10.job
Submitted batch job 27664410
Submitting: sbatch udb7_12.job
Submitted batch job 27664411
Submitting: sbatch udb7_13.job
Submitted batch job 27664412
Submitting: sbatch udb7_14.job
Submitted batch job 27664413
Submitting: sbatch udb7_15.job
Submitted batch job 27664414
Submitting: sbatch udb7_17.job
Submitted batch job 27664415
Submitting: sbatch udb7_18.job
Submitted batch job 27664416


In [8]:
submit_jobs(8, [7,10,12,13,14,15,17])

Submitting: sbatch udb8_7.job
Submitted batch job 27664417
Submitting: sbatch udb8_10.job
Submitted batch job 27664418
Submitting: sbatch udb8_12.job
Submitted batch job 27664419
Submitting: sbatch udb8_13.job
Submitted batch job 27664420
Submitting: sbatch udb8_14.job
Submitted batch job 27664421
Submitting: sbatch udb8_15.job
Submitted batch job 27664422
Submitting: sbatch udb8_17.job
Submitted batch job 27664423


In [9]:
submit_jobs(9, [7,10,12,13,14,15,17])

Submitting: sbatch udb9_7.job
Submitted batch job 27664424
Submitting: sbatch udb9_10.job
Submitted batch job 27664425
Submitting: sbatch udb9_12.job
Submitted batch job 27664426
Submitting: sbatch udb9_13.job
Submitted batch job 27664427
Submitting: sbatch udb9_14.job
Submitted batch job 27664429
Submitting: sbatch udb9_15.job
Submitted batch job 27664430
Submitting: sbatch udb9_17.job
Submitted batch job 27664431


# 3D UMap

In [ ]:
import os
import re
from pathlib import Path

def create_3dumap_scripts_batch(x, y_values, base_directory="."):
  """
  Generate 3dumap scripts from udb configurations.
  Changes n_components to 3 and only runs hyrax umap (no save_to_database).
  
  Parameters:
  -----------
  x : int
      The run number (e.g., 4 for run4)
  y_values : list
      List of job numbers to process (e.g., [1, 3, 5, 7])
  base_directory : str
      Base directory path (will append runx)
  """

  # Append runx to the base directory path
  run_dir = Path(base_directory) / f"run{x}"
  results = []

  for y in y_values:
      udb_name = f"udb{x}_{y}"
      dumap_name = f"3dumap{x}_{y}"

      # File paths
      udb_toml = run_dir / f"{udb_name}.toml"
      udb_job = run_dir / f"{udb_name}.job"

      # Check if UDB files exist
      if not udb_toml.exists():
          print(f"Warning: {udb_toml} not found, skipping...")
          continue

      # Create 3dumap config
      dumap_toml = run_dir / f"{dumap_name}.toml"
      with open(udb_toml, 'r') as f:
          config_content = f.read()

      # Change n_components from 2 to 3
      config_content = re.sub(r'n_components = 2', 'n_components = 3', config_content)

      with open(dumap_toml, 'w') as f:
          f.write(config_content)

      # Create 3dumap job script
      if udb_job.exists():
          dumap_job = run_dir / f"{dumap_name}.job"
          with open(udb_job, 'r') as f:
              job_content = f.read()

          # Update job script parameters
          job_content = re.sub(rf'--job-name=u{x}_{y}', f'--job-name=3u{x}_{y}', job_content)
          job_content = re.sub(rf'--output=udb{x}_{y}\.txt', f'--output=3dumap{x}_{y}.txt', job_content)
          job_content = re.sub(rf'udb{x}_{y}\.toml', f'3dumap{x}_{y}.toml', job_content)

          # Remove the save_to_database line, keep only the umap line
          lines = job_content.split('\n')
          filtered_lines = []
          for line in lines:
              if 'hyrax save_to_database' not in line:
                  filtered_lines.append(line)

          job_content = '\n'.join(filtered_lines)

          with open(dumap_job, 'w') as f:
              f.write(job_content)

          results.append((dumap_toml, dumap_job))
          print(f"Created 3dumap scripts for {udb_name}")
      else:
          print(f"Warning: {udb_job} not found for {udb_name}")

  return results

In [3]:
base_directory = "/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs"
create_3dumap_scripts_batch(4, [7,10,12,13,14,15,18], base_directory=base_directory)

Created 3dumap scripts for udb4_7
Created 3dumap scripts for udb4_10
Created 3dumap scripts for udb4_12
Created 3dumap scripts for udb4_13
Created 3dumap scripts for udb4_14
Created 3dumap scripts for udb4_15
Created 3dumap scripts for udb4_18


[(PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/3dumap4_7.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/3dumap4_7.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/3dumap4_10.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/3dumap4_10.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/3dumap4_12.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/3dumap4_12.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/3dumap4_13.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/3dumap4_13.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/3dumap4_14.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/3dumap4_14.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run4/3dumap4_15.toml'),
  PosixPath('/mmfs1/gsc

In [4]:
create_3dumap_scripts_batch(5, [7,10,12,13,14,15], base_directory=base_directory)

Created 3dumap scripts for udb5_7
Created 3dumap scripts for udb5_10
Created 3dumap scripts for udb5_12
Created 3dumap scripts for udb5_13
Created 3dumap scripts for udb5_14
Created 3dumap scripts for udb5_15


[(PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/3dumap5_7.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/3dumap5_7.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/3dumap5_10.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/3dumap5_10.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/3dumap5_12.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/3dumap5_12.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/3dumap5_13.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/3dumap5_13.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/3dumap5_14.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/3dumap5_14.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run5/3dumap5_15.toml'),
  PosixPath('/mmfs1/gsc

In [5]:
create_3dumap_scripts_batch(6, [7,10,12,13,14,15], base_directory=base_directory)

Created 3dumap scripts for udb6_7
Created 3dumap scripts for udb6_10
Created 3dumap scripts for udb6_12
Created 3dumap scripts for udb6_13
Created 3dumap scripts for udb6_14
Created 3dumap scripts for udb6_15


[(PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/3dumap6_7.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/3dumap6_7.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/3dumap6_10.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/3dumap6_10.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/3dumap6_12.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/3dumap6_12.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/3dumap6_13.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/3dumap6_13.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/3dumap6_14.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/3dumap6_14.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run6/3dumap6_15.toml'),
  PosixPath('/mmfs1/gsc

In [6]:
create_3dumap_scripts_batch(7, [7,10,12,13,14,15,17,18], base_directory=base_directory)

Created 3dumap scripts for udb7_7
Created 3dumap scripts for udb7_10
Created 3dumap scripts for udb7_12
Created 3dumap scripts for udb7_13
Created 3dumap scripts for udb7_14
Created 3dumap scripts for udb7_15
Created 3dumap scripts for udb7_17
Created 3dumap scripts for udb7_18


[(PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/3dumap7_7.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/3dumap7_7.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/3dumap7_10.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/3dumap7_10.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/3dumap7_12.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/3dumap7_12.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/3dumap7_13.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/3dumap7_13.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/3dumap7_14.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/3dumap7_14.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run7/3dumap7_15.toml'),
  PosixPath('/mmfs1/gsc

In [7]:
create_3dumap_scripts_batch(8, [7,10,12,13,14,15,17], base_directory=base_directory)

Created 3dumap scripts for udb8_7
Created 3dumap scripts for udb8_10
Created 3dumap scripts for udb8_12
Created 3dumap scripts for udb8_13
Created 3dumap scripts for udb8_14
Created 3dumap scripts for udb8_15
Created 3dumap scripts for udb8_17


[(PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/3dumap8_7.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/3dumap8_7.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/3dumap8_10.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/3dumap8_10.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/3dumap8_12.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/3dumap8_12.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/3dumap8_13.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/3dumap8_13.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/3dumap8_14.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/3dumap8_14.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run8/3dumap8_15.toml'),
  PosixPath('/mmfs1/gsc

In [8]:
create_3dumap_scripts_batch(9, [7,10,12,13,14,15,17], base_directory=base_directory)

Created 3dumap scripts for udb9_7
Created 3dumap scripts for udb9_10
Created 3dumap scripts for udb9_12
Created 3dumap scripts for udb9_13
Created 3dumap scripts for udb9_14
Created 3dumap scripts for udb9_15
Created 3dumap scripts for udb9_17


[(PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/3dumap9_7.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/3dumap9_7.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/3dumap9_10.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/3dumap9_10.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/3dumap9_12.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/3dumap9_12.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/3dumap9_13.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/3dumap9_13.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/3dumap9_14.toml'),
  PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/3dumap9_14.job')),
 (PosixPath('/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/run9/3dumap9_15.toml'),
  PosixPath('/mmfs1/gsc

In [5]:
import os

def submit_jobs(run_number, job_numbers):
      """
      Submit job scripts for specified job numbers.
      
      Parameters:
      -----------
      run_number : int
          The run number (e.g., 4 for run4)
      job_numbers : list
          List of job numbers to submit (e.g., [1, 3, 5, 7])
      """
      # Go to the run directory
      run_dir = f"/mmfs1/home/aritrag/aritrag_dirac/aritrag/hyrax_comcam_dp1_runs/run{run_number}"
      os.chdir(run_dir)

      # Submit each job in the array
      for i in job_numbers:
          job_file = f"3dumap{run_number}_{i}.job"
          cmd = f"sbatch {job_file}"
          print(f"Submitting: {cmd}")
          os.system(cmd)


In [11]:
submit_jobs(4, [7,10,12,13,14,15,18])

Submitting: sbatch 3dumap4_7.job
Submitted batch job 27692049
Submitting: sbatch 3dumap4_10.job
Submitted batch job 27692050
Submitting: sbatch 3dumap4_12.job
Submitted batch job 27692051
Submitting: sbatch 3dumap4_13.job
Submitted batch job 27692052
Submitting: sbatch 3dumap4_14.job
Submitted batch job 27692053
Submitting: sbatch 3dumap4_15.job
Submitted batch job 27692054
Submitting: sbatch 3dumap4_18.job
Submitted batch job 27692055


In [12]:
submit_jobs(5, [7,10,12,13,14,15])

Submitting: sbatch 3dumap5_7.job
Submitted batch job 27692372
Submitting: sbatch 3dumap5_10.job
Submitted batch job 27692373
Submitting: sbatch 3dumap5_12.job
Submitted batch job 27692374
Submitting: sbatch 3dumap5_13.job
Submitted batch job 27692375
Submitting: sbatch 3dumap5_14.job
Submitted batch job 27692376
Submitting: sbatch 3dumap5_15.job
Submitted batch job 27692377


In [6]:
submit_jobs(5, [15])

Submitting: sbatch 3dumap5_15.job
Submitted batch job 27695167


In [7]:
submit_jobs(6, [7,10,12,13,14,15])

Submitting: sbatch 3dumap6_7.job
Submitted batch job 27695169
Submitting: sbatch 3dumap6_10.job
Submitted batch job 27695170
Submitting: sbatch 3dumap6_12.job
Submitted batch job 27695171
Submitting: sbatch 3dumap6_13.job
Submitted batch job 27695172
Submitting: sbatch 3dumap6_14.job
Submitted batch job 27695173
Submitting: sbatch 3dumap6_15.job
Submitted batch job 27695174


In [8]:
submit_jobs(7, [7,10,12,13,14,15,17,18])

Submitting: sbatch 3dumap7_7.job
Submitted batch job 27695322
Submitting: sbatch 3dumap7_10.job
Submitted batch job 27695323
Submitting: sbatch 3dumap7_12.job
Submitted batch job 27695324
Submitting: sbatch 3dumap7_13.job
Submitted batch job 27695325
Submitting: sbatch 3dumap7_14.job
Submitted batch job 27695326
Submitting: sbatch 3dumap7_15.job
Submitted batch job 27695327
Submitting: sbatch 3dumap7_17.job
Submitted batch job 27695328
Submitting: sbatch 3dumap7_18.job
Submitted batch job 27695329


In [9]:
submit_jobs(8, [7,10,12,13,14,15,17])

Submitting: sbatch 3dumap8_7.job
Submitted batch job 27695330
Submitting: sbatch 3dumap8_10.job
Submitted batch job 27695331
Submitting: sbatch 3dumap8_12.job
Submitted batch job 27695332
Submitting: sbatch 3dumap8_13.job
Submitted batch job 27695333
Submitting: sbatch 3dumap8_14.job
Submitted batch job 27695334
Submitting: sbatch 3dumap8_15.job
Submitted batch job 27695335
Submitting: sbatch 3dumap8_17.job
Submitted batch job 27695336


In [17]:
submit_jobs(8, [10])

Submitting: sbatch 3dumap8_10.job
Submitted batch job 27696267


In [13]:
submit_jobs(9, [7,10,12,13,14,15,17])

Submitting: sbatch 3dumap9_7.job
Submitted batch job 27695428
Submitting: sbatch 3dumap9_10.job
Submitted batch job 27695429
Submitting: sbatch 3dumap9_12.job
Submitted batch job 27695436
Submitting: sbatch 3dumap9_13.job
Submitted batch job 27695437
Submitting: sbatch 3dumap9_14.job
Submitted batch job 27695438
Submitting: sbatch 3dumap9_15.job
Submitted batch job 27695439
Submitting: sbatch 3dumap9_17.job
Submitted batch job 27695440


In [10]:
import os
import re
import sys
from pathlib import Path
import toml


# Add the hyrax module to the path so we can import save_umap_json
sys.path.append('/mmfs1/home/aritrag/aritrag_dirac/aritrag/repos/hyrax/src')

# Import using importlib since the module name starts with a number
import importlib.util
spec = importlib.util.spec_from_file_location("save_umap_to_json",
"/mmfs1/home/aritrag/aritrag_dirac/aritrag/repos/hyrax/src/hyrax/3d_viz/save_umap_to_json.py")
save_umap_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(save_umap_module)
save_umap_json = save_umap_module.save_umap_json

def extract_umap_directory(dumap_output_file):
  """Extract UMAP directory from 3dumap output file."""
  try:
      with open(dumap_output_file, 'r') as f:
          content = f.read()

      # Look for the UMAP results save pattern
      pattern = r'Saving UMAP results to (.+)'
      match = re.search(pattern, content)

      if match:
          umap_dir = match.group(1).strip()
          return umap_dir
      else:
          raise ValueError(f"Could not find UMAP directory in {dumap_output_file}")

  except FileNotFoundError:
      raise FileNotFoundError(f"3dumap output file not found: {dumap_output_file}")

def extract_astropy_table_path(dumap_config_file):
  """Extract astropy_table path from 3dumap config file."""
  try:
      with open(dumap_config_file, 'r') as f:
          config = toml.load(f)

      astropy_table_path = config.get('data_set', {}).get('astropy_table')
      if astropy_table_path is None:
          raise ValueError(f"Could not find astropy_table in {dumap_config_file}")

      return astropy_table_path

  except FileNotFoundError:
      raise FileNotFoundError(f"3dumap config file not found: {dumap_config_file}")

def create_3d_viz_json(x, y, base_directory="."):
  """
  Create 3D visualization JSON file from 3dumap results.
  
  Parameters:
  -----------
  x : int
      The run number (e.g., 4 for run4)
  y : int
      The job number (e.g., 10 for 3dumap4_10)
  base_directory : str
      Base directory path (will append runx)
  """

  # Construct paths
  run_dir = Path(base_directory) / f"run{x}"
  dumap_name = f"3dumap{x}_{y}"

  dumap_output = run_dir / f"{dumap_name}.txt"
  dumap_config = run_dir / f"{dumap_name}.toml"

  # Check if files exist
  if not dumap_output.exists():
      raise FileNotFoundError(f"3dumap output file not found: {dumap_output}")
  if not dumap_config.exists():
      raise FileNotFoundError(f"3dumap config file not found: {dumap_config}")

  # Extract required paths
  try:
      umap_results_dir = extract_umap_directory(dumap_output)
      fits_table_path = extract_astropy_table_path(dumap_config)
  except (ValueError, FileNotFoundError) as e:
      print(f"Error processing {dumap_name}: {e}")
      return None

  # Create output directory if it doesn't exist
  viz_dir = Path(base_directory) / "3d_viz_files"
  viz_dir.mkdir(exist_ok=True)

  # Set output JSON path
  output_json = viz_dir / f"umap{x}_{y}.json"

  # Call save_umap_json function
  try:
      print(f"Creating 3D visualization JSON for {dumap_name}...")
      print(f"  UMAP results dir: {umap_results_dir}")
      print(f"  FITS table path: {fits_table_path}")
      print(f"  Output JSON: {output_json}")

      save_umap_json(
          results_dir=umap_results_dir,
          output_json=str(output_json),
          fits_table_path=fits_table_path,
          id_column="objectId"
      )

      print(f"Successfully created {output_json}")
      return str(output_json)

  except Exception as e:
      print(f"Error creating JSON for {dumap_name}: {e}")
      return None

def create_3d_viz_json_batch(x, y_values, base_directory="."):
  """
  Create 3D visualization JSON files for multiple y values.
  
  Parameters:
  -----------
  x : int
      The run number (e.g., 4 for run4)
  y_values : list
      List of job numbers to process (e.g., [1, 3, 5, 7])
  base_directory : str
      Base directory path (will append runx)
  """

  results = []
  failed = []

  for y in y_values:
      try:
          output_json = create_3d_viz_json(x, y, base_directory)
          if output_json:
              results.append(output_json)
          else:
              failed.append(y)
      except Exception as e:
          print(f"Failed to process 3dumap{x}_{y}: {e}")
          failed.append(y)

  # Summary
  print(f"\n3D Visualization JSON Generation Summary:")
  print(f"  Successfully created: {len(results)} JSON files")
  print(f"  Failed: {len(failed)} files")

  if failed:
      print(f"  Failed job numbers: {failed}")

  return results

In [3]:
base_directory = "/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs"
create_3d_viz_json_batch(4,[7,10,12,13,14,15,18],base_directory=base_directory)

Creating 3D visualization JSON for 3dumap4_7...
  UMAP results dir: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/results/20250718-112540-umap-8bnn
  FITS table path: /mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_20.fits
  Output JSON: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap4_7.json


Formatting JSON: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 3391/3391 [00:00<00:00, 34193.90it/s]


Successfully created /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap4_7.json
Creating 3D visualization JSON for 3dumap4_10...
  UMAP results dir: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/results/20250718-112617-umap-geFX
  FITS table path: /mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_20.fits
  Output JSON: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap4_10.json


Formatting JSON: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 3391/3391 [00:00<00:00, 33473.25it/s]


Successfully created /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap4_10.json
Creating 3D visualization JSON for 3dumap4_12...
  UMAP results dir: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/results/20250718-112617-umap-ZqK0
  FITS table path: /mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_20.fits
  Output JSON: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap4_12.json


...

Formatting JSON: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 3391/3391 [00:00<00:00, 32235.65it/s]


Successfully created /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap4_15.json
Creating 3D visualization JSON for 3dumap4_18...
  UMAP results dir: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/results/20250718-112847-umap-gwWT
  FITS table path: /mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_20.fits
  Output JSON: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap4_18.json


Formatting JSON: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 3391/3391 [00:00<00:00, 32206.53it/s]


Successfully created /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap4_18.json

3D Visualization JSON Generation Summary:
  Successfully created: 7 JSON files
  Failed: 0 files


['/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap4_7.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap4_10.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap4_12.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap4_13.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap4_14.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap4_15.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap4_18.json']

In [11]:
create_3d_viz_json_batch(5,[7,10,12,13,14,15],base_directory=base_directory)

Creating 3D visualization JSON for 3dumap5_7...
  UMAP results dir: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/results/20250718-113021-umap-XPIq
  FITS table path: /mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_20_22.fits
  Output JSON: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap5_7.json


Formatting JSON: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 15932/15932 [00:00<00:00, 34595.22it/s]


Successfully created /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap5_7.json
Creating 3D visualization JSON for 3dumap5_10...
  UMAP results dir: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/results/20250718-113021-umap-aWD0
  FITS table path: /mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_20_22.fits
  Output JSON: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap5_10.json


Formatting JSON: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 15932/15932 [00:00<00:00, 34758.02it/s]


Successfully created /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap5_10.json
Creating 3D visualization JSON for 3dumap5_12...
  UMAP results dir: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/results/20250718-113021-umap-pXxQ
  FITS table path: /mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_20_22.fits
  Output JSON: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap5_12.json


...

Formatting JSON: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 15932/15932 [00:00<00:00, 34593.47it/s]


Successfully created /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap5_14.json
Creating 3D visualization JSON for 3dumap5_15...
  UMAP results dir: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/results/20250718-123049-umap-oRtn
  FITS table path: /mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_20_22.fits
  Output JSON: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap5_15.json


Formatting JSON: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 15932/15932 [00:00<00:00, 34585.93it/s]


Successfully created /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap5_15.json

3D Visualization JSON Generation Summary:
  Successfully created: 6 JSON files
  Failed: 0 files


['/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap5_7.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap5_10.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap5_12.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap5_13.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap5_14.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap5_15.json']

In [12]:
create_3d_viz_json_batch(6,[7,10,12,13,14,15],base_directory=base_directory)

Creating 3D visualization JSON for 3dumap6_7...
  UMAP results dir: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/results/20250718-123049-umap-TFVu
  FITS table path: /mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_22_24.fits
  Output JSON: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap6_7.json


Formatting JSON: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 73564/73564 [00:02<00:00, 34660.89it/s]


Successfully created /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap6_7.json
Creating 3D visualization JSON for 3dumap6_10...
  UMAP results dir: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/results/20250718-122938-umap-R3X8
  FITS table path: /mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_22_24.fits
  Output JSON: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap6_10.json


Formatting JSON: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 73564/73564 [00:02<00:00, 34753.12it/s]


Successfully created /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap6_10.json
Creating 3D visualization JSON for 3dumap6_12...
  UMAP results dir: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/results/20250718-122931-umap-Tmpp
  FITS table path: /mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_22_24.fits
  Output JSON: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap6_12.json


...

Formatting JSON: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 73564/73564 [00:02<00:00, 34737.73it/s]


Successfully created /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap6_14.json
Creating 3D visualization JSON for 3dumap6_15...
  UMAP results dir: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/results/20250718-123841-umap-GvWO
  FITS table path: /mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_22_24.fits
  Output JSON: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap6_15.json


Formatting JSON: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 73564/73564 [00:02<00:00, 34271.67it/s]


Successfully created /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap6_15.json

3D Visualization JSON Generation Summary:
  Successfully created: 6 JSON files
  Failed: 0 files


['/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap6_7.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap6_10.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap6_12.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap6_13.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap6_14.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap6_15.json']

In [15]:
create_3d_viz_json_batch(7,[7,10,12,13,14,15,17,18],base_directory=base_directory)

Creating 3D visualization JSON for 3dumap7_7...
  UMAP results dir: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/results/20250718-125410-umap-cZBE
  FITS table path: /mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_20.fits
  Output JSON: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap7_7.json


Formatting JSON: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 3391/3391 [00:00<00:00, 34709.86it/s]


Successfully created /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap7_7.json
Creating 3D visualization JSON for 3dumap7_10...
  UMAP results dir: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/results/20250718-125342-umap-L2Ju
  FITS table path: /mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_20.fits
  Output JSON: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap7_10.json


Formatting JSON: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 3391/3391 [00:00<00:00, 34596.05it/s]


Successfully created /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap7_10.json
Creating 3D visualization JSON for 3dumap7_12...
  UMAP results dir: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/results/20250718-125323-umap-Nu6E
  FITS table path: /mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_20.fits
  Output JSON: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap7_12.json


...

Formatting JSON: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 3391/3391 [00:00<00:00, 34034.26it/s]


Successfully created /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap7_17.json
Creating 3D visualization JSON for 3dumap7_18...
  UMAP results dir: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/results/20250718-125720-umap-deRv
  FITS table path: /mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_20.fits
  Output JSON: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap7_18.json


Formatting JSON: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████| 3391/3391 [00:00<00:00, 34478.38it/s]


Successfully created /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap7_18.json

3D Visualization JSON Generation Summary:
  Successfully created: 8 JSON files
  Failed: 0 files


['/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap7_7.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap7_10.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap7_12.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap7_13.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap7_14.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap7_15.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap7_17.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap7_18.json']

In [18]:
create_3d_viz_json_batch(8,[7,10,12,13,14,15,17],base_directory=base_directory)

Creating 3D visualization JSON for 3dumap8_7...
  UMAP results dir: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/results/20250718-125634-umap-MwEb
  FITS table path: /mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_20_22.fits
  Output JSON: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap8_7.json


Formatting JSON: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 15932/15932 [00:00<00:00, 34900.94it/s]


Successfully created /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap8_7.json
Creating 3D visualization JSON for 3dumap8_10...
  UMAP results dir: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/results/20250718-132316-umap-rl2O
  FITS table path: /mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_20_22.fits
  Output JSON: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap8_10.json


Formatting JSON: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 15932/15932 [00:00<00:00, 35514.87it/s]


Successfully created /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap8_10.json
Creating 3D visualization JSON for 3dumap8_12...
  UMAP results dir: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/results/20250718-125816-umap-589S
  FITS table path: /mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_20_22.fits
  Output JSON: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap8_12.json


...

Formatting JSON: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 15932/15932 [00:00<00:00, 35596.64it/s]


Successfully created /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap8_15.json
Creating 3D visualization JSON for 3dumap8_17...
  UMAP results dir: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/results/20250718-130140-umap-7qAB
  FITS table path: /mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_20_22.fits
  Output JSON: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap8_17.json


Formatting JSON: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 15932/15932 [00:00<00:00, 35379.90it/s]


Successfully created /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap8_17.json

3D Visualization JSON Generation Summary:
  Successfully created: 7 JSON files
  Failed: 0 files


['/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap8_7.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap8_10.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap8_12.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap8_13.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap8_14.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap8_15.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap8_17.json']

In [19]:
create_3d_viz_json_batch(9,[7,10,12,13,14,15,17],base_directory=base_directory)

Creating 3D visualization JSON for 3dumap9_7...
  UMAP results dir: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/results/20250718-130256-umap-kRSE
  FITS table path: /mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_22_24.fits
  Output JSON: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap9_7.json


Formatting JSON: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 73564/73564 [00:02<00:00, 35944.03it/s]


Successfully created /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap9_7.json
Creating 3D visualization JSON for 3dumap9_10...
  UMAP results dir: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/results/20250718-130401-umap-hzaZ
  FITS table path: /mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_22_24.fits
  Output JSON: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap9_10.json


Formatting JSON: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 73564/73564 [00:02<00:00, 35264.70it/s]


Successfully created /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap9_10.json
Creating 3D visualization JSON for 3dumap9_12...
  UMAP results dir: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/results/20250718-130346-umap-MjX8
  FITS table path: /mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_22_24.fits
  Output JSON: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap9_12.json


...

Formatting JSON: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 73564/73564 [00:02<00:00, 34719.11it/s]


Successfully created /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap9_15.json
Creating 3D visualization JSON for 3dumap9_17...
  UMAP results dir: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/results/20250718-131157-umap-vgA3
  FITS table path: /mmfs1/gscratch/escience/aritrag/comcam_dp1/catalogs/eaabdm_22_24.fits
  Output JSON: /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap9_17.json


Formatting JSON: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████| 73564/73564 [00:02<00:00, 34689.09it/s]


Successfully created /mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap9_17.json

3D Visualization JSON Generation Summary:
  Successfully created: 7 JSON files
  Failed: 0 files


['/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap9_7.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap9_10.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap9_12.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap9_13.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap9_14.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap9_15.json',
 '/mmfs1/gscratch/dirac/aritrag/hyrax_comcam_dp1_runs/3d_viz_files/umap9_17.json']